# YOLO v3 TF2 GPU Colab
- You only look once!!!

### 1. Clone and install dependencies
- 설치가 끝나면 '런타임 다시시작'하여야 함

In [ ]:
# 설치되어있는 tensorflow, keras 삭제
!pip uninstall -y tensorflow keras

In [ ]:
!git clone https://github.com/zzh8829/yolov3-tf2

In [ ]:
# yolov3-tf2로 이동
%cd yolov3-tf2

In [ ]:
!pip install -r requirements-gpu.txt

In [ ]:
import tensorflow as tf
tf.__version__

In [ ]:
!pip install tensorflow==2.4.2

In [ ]:
!pip install keras==2.4.2

In [ ]:
# 런타임 다시시작

### 2. Tensorflow2 version 확인

In [ ]:
%cd yolov3-tf2/
!ls

In [ ]:
import tensorflow as tf
tf.__version__

### 3. Darknet의 pretrained weight을 변환

In [ ]:
!wget https://pjreddie.com/media/files/yolov3.weights -O data/yolov3.weights

In [ ]:
!python convert.py --weights ./data/yolov3.weights --output ./checkpoints/yolov3.tf

### 4. Initialize detector

In [6]:
import sys
from absl import app, logging, flags
from absl.flags import FLAGS
import time
import cv2
import numpy as np
import tensorflow as tf
from yolov3_tf2.models import YoloV3, YoloV3Tiny
from yolov3_tf2.dataset import transform_images, load_tfrecord_dataset
from yolov3_tf2.utils import draw_outputs

from tensorflow.keras.preprocessing import image
import matplotlib.pyplot as plt
from IPython.display import Image, display

In [ ]:
!ls -l checkpoints

In [ ]:
flags.DEFINE_string('classes', './data/coco.names', 'path to classes file')
flags.DEFINE_string('weights', './checkpoints/yolov3.tf', 'path to weights file')
flags.DEFINE_boolean('tiny', False, 'yolov3 or yolov3-tiny')
flags.DEFINE_integer('size', 416, 'resize images to')
flags.DEFINE_string('image', './data/girl.png', 'path to input image')
flags.DEFINE_string('tfrecord', None, 'tfrecord instead of image')
flags.DEFINE_string('output', './output.jpg', 'path to output image')
flags.DEFINE_integer('num_classes', 80, 'number of classes in the model')

app._run_init(['yolov3'], app.parse_flags_with_usage)

#physical_devices = tf.config.experimental.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)

### 5. Detect Image

In [ ]:
yolo = YoloV3(classes=FLAGS.num_classes)
yolo.load_weights(FLAGS.weights).expect_partial()
class_names = [c.strip() for c in open(FLAGS.classes).readlines()]
for index, name in enumerate(class_names):
    print(name, end=', ')
    if (index+1) % 10 == 0:
        print()

In [ ]:
len(class_names)

In [11]:
def detector():
    img_raw = tf.image.decode_image(open(FLAGS.image,'rb').read(), channels=3)
    img = tf.expand_dims(img_raw, 0)
    img = transform_images(img, FLAGS.size)

    t1 = time.time()
    boxes, scores, classes, nums = yolo(img)
    t2 = time.time()
    print(f'time: {t2 - t1}')

    for i in range(nums[0]):
        print(f'detections: {class_names[int(classes[0][i])]}, {np.array(scores[0][i])}, {np.array(boxes[0][i])}')

    img = cv2.cvtColor(img_raw.numpy(), cv2.COLOR_RGB2BGR)
    img = draw_outputs(img, (boxes, scores, classes, nums), class_names)

    return img

In [ ]:
FLAGS.image = 'data/girl.png'
Image(filename=FLAGS.image, width=500)

In [ ]:
output = detector()
display(Image(data=bytes(cv2.imencode('.jpg', output)[1]), width=500))

In [ ]:
FLAGS.image = 'data/meme.jpg'
Image(filename=FLAGS.image, width=800)

In [ ]:
output = detector()
display(Image(data=bytes(cv2.imencode('.jpg', output)[1]), width=800))

In [ ]:
FLAGS.image = 'data/street.jpg'
Image(filename=FLAGS.image, width=1000)

In [ ]:
output = detector()
display(Image(data=bytes(cv2.imencode('.jpg', output)[1]), width=1000))